In [ ]:
from zipfile import ZipFile
import re
from datetime import date, timedelta
from games import games, symbols, MAX, MEDALS

def calcScore(board, max):
    score = 0
    for s in symbols.keys():
      count = board.count(s)
      score += count * min(max, symbols[s])
    return score

chats = ""
with ZipFile("../data/export.zip", 'r') as zf: 
    ba = zf.read("_chat.txt")
    chats = ba.decode()
    zf.close() 

chats = chats.replace(" 🎉", "").replace("🙂 ", "").replace("🥕 ", "").replace("🌀 ", "")

pattern = r"\[\d+\/\d+\/\d+, \d+:\d+:\d+\] ([A-Za-z é&]+): \s*([ ([A-Za-z 🥕]+) #?(\d,?\d+)( (\d)\/(\d))?([^\[]+)"
matches = re.findall(pattern, chats)
plays = []

for x in matches:
    name = x[0].split()[0]   
    game_name = x[1]
    keys = [key for key, val in games.items() if key in game_name]
    if len(keys) > 0:
        game_key = keys[0]
        game = games[game_key]
        day = int(x[2].replace(",", ""))
        game_date = game["day"] + timedelta(days=day)
        board = ""
        turns = MAX
        if game["useBoard"]:
            board = x[6]
            turns = calcScore(board, game["max"])
        elif x[4]:
            turns = int(x[4])
        plays.append({"date":game_date, "person": name, "heading": game["heading"], "game": game_key, "turns": turns})

# groups plays by date and person
grouped = {}
for p in plays:
    key = p["date"]
    game = p["game"]
    if key in grouped:
        game = grouped[key][game]
        game.append(p)
    else:
        day_games = {key: [] for key in games.keys()}
        day_games[game].append(p)
        grouped[key] = day_games

for day in grouped.keys():
    day_games = grouped[day]
    for game_key in day_games.keys():
        day_plays = day_games[game_key]
        sorted_plays = sorted(day_plays, key=lambda x: x["turns"])
        medal_count = 0;
        medal_index = -1;
        last_turn = 0;
        for p in sorted_plays:
            turns = p["turns"]
            if turns > last_turn:
                if medal_index >= 3:
                    break
                p["medal"] = MEDALS[medal_index]
            last_turn = turns
            medal_index += 1

yesterday = date.today() - timedelta(days=1)
grouped[yesterday]


{'Wordle': [{'date': datetime.date(2025, 8, 25),
   'person': 'Neil',
   'heading': 'W',
   'game': 'Wordle',
   'turns': 6},
  {'date': datetime.date(2025, 8, 25),
   'person': 'Pete',
   'heading': 'W',
   'game': 'Wordle',
   'turns': 3,
   'medal': 1},
  {'date': datetime.date(2025, 8, 25),
   'person': 'Buzz',
   'heading': 'W',
   'game': 'Wordle',
   'turns': 5,
   'medal': 100},
  {'date': datetime.date(2025, 8, 25),
   'person': 'Alan',
   'heading': 'W',
   'game': 'Wordle',
   'turns': 5},
  {'date': datetime.date(2025, 8, 25),
   'person': 'Michael',
   'heading': 'W',
   'game': 'Wordle',
   'turns': 3}],
 'Wortel': [],
 'Daily Quordle': [{'date': datetime.date(2025, 8, 25),
   'person': 'Neil',
   'heading': 'Q',
   'game': 'Daily Quordle',
   'turns': 21,
   'medal': 10000},
  {'date': datetime.date(2025, 8, 25),
   'person': 'Buzz',
   'heading': 'Q',
   'game': 'Daily Quordle',
   'turns': 21},
  {'date': datetime.date(2025, 8, 25),
   'person': 'Pete',
   'heading': '